In [1]:
import open3d as o3d
import numpy as np
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import multiprocessing as mp
from multiprocessing import Pool
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten

In [2]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])

In [3]:
def create_cuboid(plane_eq):
    # Extract the coefficients A, B, C, and D from the plane equations
    A1, B1, C1, D1 = plane_eq[0]
    A2, B2, C2, D2 = plane_eq[1]
    A3, B3, C3, D3 = plane_eq[2]

    # Calculate the intersection lines between the planes
    line1 = np.cross([A1, B1, C1], [A2, B2, C2])
    line2 = np.cross([A2, B2, C2], [A3, B3, C3])
    line3 = np.cross([A3, B3, C3], [A1, B1, C1])

    # Calculate the corner points of the cuboid
    corner_points = []
    for i in range(2):
        for j in range(2):
            for k in range(2):
                corner_point = (
                    line1 * i / np.linalg.norm(line1) +
                    line2 * j / np.linalg.norm(line2) +
                    line3 * k / np.linalg.norm(line3)
                )
                corner_points.append(corner_point)

    return corner_points

In [4]:
points = np.asarray(pcd.points) # Load your point cloud as a numpy array (N, 3)

cube1 = pyrsc.Cuboid()
best_eq, best_inliers = cube1.fit(points, thresh=0.1, maxIteration=5000)
box_points = create_cuboid(best_eq)

o3d_box_points = o3d.utility.Vector3dVector(box_points)


In [5]:
print(best_eq)
test = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_box_points)
test.color = (1,0,0)
part_pcd = pcd.select_by_index(best_inliers)
part_pcd.paint_uniform_color((1,0,0))

o3d.visualization.draw_geometries([pcd, part_pcd, test])

In [6]:
oboxes = pcd.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=1,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    geometries.append(mesh)
    geometries.append(obox)

#o3d.visualization.draw_geometries(geometries + [boundarys.to_legacy()])
o3d.visualization.draw_geometries([pcd] + geometries)

In [7]:
def filter_by_normal(pcd):
    ind = []
    for i in range(len(pcd.points)):
        if ((np.abs(pcd.normals[i][2]) < 0.3) and ((np.abs(pcd.normals[i][0]) > 0.8) or (np.abs(pcd.normals[i][1]) > 0.8))):
            pcd.points[i][2] = np.random.rand()*0.1
            pass
        else:
            ind.append(i)
    result = pcd.select_by_index(ind, invert=True)
    return result

In [8]:
filtered_pcd = filter_by_normal(pcd)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(filtered_pcd, 0.1)
o3d.visualization.draw_geometries([voxel_grid])

#print(np.asarray(filtered_pcd.points[0:10]))


In [9]:
def create_box_at_point(point):
    
    box = o3d.geometry.TriangleMesh.create_box(0.1,0.1,0.1)
    box.paint_uniform_color([0,1,0])
    
    box.translate(point, False)
    
    
    return box
    

In [10]:
def get_points_on_line(point1, point2, distance):
    # Calculate the direction vector of the line
    direction_vector = point2 - point1

    # Calculate the length of the line
    line_length = np.linalg.norm(direction_vector)

    # Calculate the number of points based on the given distance
    num_points = int(line_length / distance) + 1

    # Calculate the step size between each point
    if (num_points <= 1):
        num_points += 1
    step_size = 1.0 / (num_points - 1)

    # Calculate the parameter values for each point on the line
    parameters = np.arange(0, 1 + step_size, step_size)

    # Calculate the points on the line using the parameter values
    points = [point1 + parameter * direction_vector for parameter in parameters]

    return points

In [11]:
n_points = len(filtered_pcd.points)
line_points = [(np.random.randint(n_points), np.random.randint(n_points)) for i in range(0,n_points,10)]
lineset = o3d.geometry.LineSet.create_from_point_cloud_correspondences(filtered_pcd, filtered_pcd, line_points)

#o3d.visualization.draw_geometries([lineset])
boxes = []
indexes = []
intersection_points = []
int_points = o3d.geometry.PointCloud()
for i in range(len(line_points)):
    p1 = lineset.get_line_coordinate(i)[0]
    p2 = lineset.get_line_coordinate(i)[1]
  
    linepoints = get_points_on_line(p1,p2,0.1)

    lp_o3d = o3d.utility.Vector3dVector(linepoints)

    is_included = voxel_grid.check_if_included(lp_o3d)

    
    for bl,x in zip(is_included, range(len(is_included))):
        if bl:
            indexes.append(voxel_grid.get_voxel(lp_o3d[x]))

    intersection_points.append([linepoints[x] for x in range(len(linepoints))  if not  is_included[x]] )

intersection_points = list(deepflatten(intersection_points, depth=1))

int_points.points.extend(o3d.utility.Vector3dVector(intersection_points))  
   

o3d.visualization.draw_geometries([voxel_grid] + [int_points])

In [12]:
def angle_between_vectors(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    # Normalize the vectors to unit length
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    
    return np.degrees(angle)

In [13]:
def are_vectors_perpendicular(v1, v2, threshold):
    
    # Normalize the vectors to unit length
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    angle = np.degrees(angle)
    
    if (180 <= angle < 360):
        angle -= 180
        
    if ((90-threshold) <= angle <= (90+threshold)):
        return True
    else:
        return False


In [29]:
oboxes = filtered_pcd.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=1,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
meshes = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    meshes.append(mesh)
    geometries.append(obox)

#o3d.visualization.draw_geometries(geometries + [boundarys.to_legacy()])
o3d.visualization.draw_geometries(meshes)
#for obox, mesh in zip(oboxes, meshes):
 #   print(obox.extent)
  #  print(mesh.vertex_normals[0])

In [15]:
def devide_meshes_hor_ver(meshes):

    first = meshes[0]

    hor_patches = []
    ver_patches = [first]

    for i in range(1,len(meshes)):
        patch_normal = meshes[i].vertex_normals[0]

        if (are_vectors_perpendicular(first.vertex_normals[0], patch_normal, 15)):
            hor_patches.append(meshes[i])
        else:
            ver_patches.append(meshes[i])



    return hor_patches, ver_patches
hor_patches, ver_patches = devide_meshes_hor_ver(meshes)
o3d.visualization.draw_geometries(ver_patches)
o3d.visualization.draw_geometries(hor_patches)
print(len(hor_patches), len(ver_patches))

In [16]:
def sample_multiple_meshes(meshes):
    pc = o3d.geometry.PointCloud()

    for mesh in meshes:
        part = mesh.sample_points_poisson_disk(number_of_points=2000)
        pc.points.extend(part.points)
    return pc

In [17]:
hor_pointcloud = sample_multiple_meshes(hor_patches)
ver_pointcloud = sample_multiple_meshes(ver_patches)



In [18]:
o3d.visualization.draw_geometries([hor_pointcloud, ver_pointcloud]+ver_patches+hor_patches)
o3d.visualization.draw_geometries([ver_pointcloud]+ver_patches)

In [19]:
def get_mesh_distance(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    bb1 = mesh1.get_oriented_bounding_box()
    bb1_center = bb1.get_center()[1-o[orientation]]
    bb2 = mesh2.get_oriented_bounding_box()
    bb2_center = bb2.get_center()[1-o[orientation]]
    
    
    if (bb1_center < 0 < bb2_center):
        dist = -bb1_center + bb2_center
    elif (bb2_center < 0 < bb1_center):
        dist = bb1_center - bb2_center
    else:
        dist = np.abs(bb1_center-bb2_center)
        
    return dist

In [20]:
def find_nearest_mesh(mesh1, meshes, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    dist = np.Inf
    index = 0
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb1_center = bb1.get_center()[1-o[orientation]]
    
    for mesh2, i in zip(meshes, range(len(meshes))):
        
        bb2 = mesh2.get_oriented_bounding_box()
        bb2_center = bb2.get_center()[1-o[orientation]]
        
        if not (mesh_correspondance2(mesh1, mesh2, orientation)):
            continue
        
        
        if (bb1_center < 0 < bb2_center):
            dist_tmp = -bb1_center + bb2_center
        elif (bb2_center < 0 < bb1_center):
            dist_tmp = bb1_center - bb2_center
        else:
            dist_tmp = np.abs(bb1_center-bb2_center)

        if dist_tmp == 0:
            continue
        if dist_tmp < dist:
            dist = dist_tmp
            index = i
    
    return dist, index
    
    

In [21]:
def mesh_correspondance(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb2 = mesh2.get_oriented_bounding_box()
    bb1_min = bb1.get_min_bound()
    bb1_max = bb1.get_max_bound()
    bb2_min = bb2.get_min_bound()
    bb2_max = bb2.get_max_bound()
    
    
    if (bb1_min[o[orientation]] < bb2_min[o[orientation]] < bb1_max[o[orientation]]):
        return True
    if (bb1_min[o[orientation]] < bb2_max[o[orientation]] < bb1_max[o[orientation]]):
        return True
    if (bb2_min[o[orientation]] < bb1_min[o[orientation]] < bb2_max[o[orientation]]):
        return True
    if (bb2_min[o[orientation]] < bb1_max[o[orientation]] < bb2_max[o[orientation]]):
        return True
    
    return False
    

In [22]:
def mesh_correspondance2(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb2 = mesh2.get_oriented_bounding_box()
    bb1_min = bb1.get_min_bound()
    bb1_max = bb1.get_max_bound()
    bb2_min = bb2.get_min_bound()
    bb2_max = bb2.get_max_bound()
    bb1_center = bb1.get_center()[o[orientation]]
    bb2_center = bb2.get_center()[o[orientation]]
    
    bb1_center2 = bb1.get_center()[1-o[orientation]]
    bb2_center2 = bb2.get_center()[1-o[orientation]]
    
    if (bb1_center2 < 0 < bb2_center2):
        dist = -bb1_center2 + bb2_center2
    elif (bb2_center2 < 0 < bb1_center2):
        dist = bb1_center2 - bb2_center2
    else:
        dist = np.abs(bb1_center2-bb2_center2)
    
    if (dist < 1):
        return False
    #print("dist between planes: ", np.abs(bb1_center-bb2_center))
    if (bb1_min[o[orientation]] < bb2_center < bb1_max[o[orientation]]):
        return True
    if (bb2_min[o[orientation]] < bb1_center < bb2_max[o[orientation]]):

        return True
    
    
    return False
    

In [31]:
corr = []

    
for i in range(len(hor_patches)):
    mesh1 = hor_patches[i]
    for j in range(i+1, len(hor_patches)):
        mesh2 = hor_patches[j]
        nearest_dist, _  = find_nearest_mesh(mesh1, hor_patches, "horizontal")
        
      
        if (nearest_dist < get_mesh_distance(mesh1, mesh2, "horizontal")):
            continue
        
        if (mesh_correspondance2(mesh1, mesh2, "horizontal")):
            
            part1 = mesh1.sample_points_poisson_disk(number_of_points=2000)
            part2 = mesh2.sample_points_poisson_disk(number_of_points=2000)

            n_points1 = len(part1.points)
            n_points2 = len(part2.points)
            line_points1 = [np.random.randint(n_points1) for i in range(0,n_points1,10)]
            line_points2 = [np.random.randint(n_points2) for i in range(0,n_points2,10)]
            lineset1 = o3d.geometry.LineSet.create_from_point_cloud_correspondences(part1, part2, list(zip(line_points1, line_points2)))
            lineset1.paint_uniform_color(np.random.rand(3))
            corr.append(lineset1)
o3d.visualization.draw_geometries(corr)

In [32]:

    
for i in range(len(ver_patches)):
    mesh1 = ver_patches[i]
    for j in range(i+1, len(ver_patches)):
        mesh2 = ver_patches[j]
        
        nearest_dist, _  = find_nearest_mesh(mesh1, ver_patches, "vertical")
        
        
        if (nearest_dist < get_mesh_distance(mesh1, mesh2, "vertical")):
            
            continue
            
        
        print("nearest: ", nearest_dist)
        print("other: ", get_mesh_distance(mesh1, mesh2, "vertical"))
        if (mesh_correspondance2(mesh1, mesh2, "vertical")):
            #print(mesh1.get_oriented_bounding_box().center)
            #print(mesh2.get_oriented_bounding_box().center)
            part1 = mesh1.sample_points_poisson_disk(number_of_points=2000)
            part2 = mesh2.sample_points_poisson_disk(number_of_points=2000)

            n_points1 = len(part1.points)
            n_points2 = len(part2.points)
            line_points1 = [np.random.randint(n_points1) for i in range(0,n_points1,10)]
            line_points2 = [np.random.randint(n_points2) for i in range(0,n_points2,10)]
            lineset1 = o3d.geometry.LineSet.create_from_point_cloud_correspondences(part1, part2, list(zip(line_points1, line_points2)))
            lineset1.paint_uniform_color(np.random.rand(3))
            
            corr.append(lineset1)
o3d.visualization.draw_geometries(corr+ver_patches)

In [33]:
o3d.visualization.draw_geometries(corr + [filtered_pcd])

In [36]:
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        filtered_pcd.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([filtered_pcd])